<a href="https://colab.research.google.com/github/BatoolMM/DeepSulfation/blob/master/Pre_process_and_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook
!pip install fastcore==1.0.9
!pip install utils

In [2]:
import pandas as pd
import csv
import numpy as np
import keras.utils.np_utils as kutils
import matplotlib.pyplot as plt
from utils import *
import fastbook
from fastbook import *
from fastai.vision.all import *
from fastai.vision.widgets import ImageClassifierCleaner

In [12]:
file = '/content/CG-20-306_SD1.xlsx'
training = pd.read_excel(file, sheet_name=0)
test =pd.read_excel(file, sheet_name=1)

In [13]:
training.head(2)

,POSITIVE,Unnamed: 1,Unnamed: 2
0,P14543,ATTNGVVPADVILGTEDGAEYDDEDEDYDLATTRLGLEDVG,2.0
1,P14543,PADVILGTEDGAEYDDEDEDYDLATTRLGLEDVGTTPFSYK,2.0


In [14]:
removed_row = training['POSITIVE'].eq('NEGATIVE')
mask = removed_row.cumsum().eq(0)
training['Target'] = np.where(mask, '1', '0')
training = training[~removed_row].copy()
training.head(2)


,POSITIVE,Unnamed: 1,Unnamed: 2,Target
0,P14543,ATTNGVVPADVILGTEDGAEYDDEDEDYDLATTRLGLEDVG,2.0,1
1,P14543,PADVILGTEDGAEYDDEDEDYDLATTRLGLEDVGTTPFSYK,2.0,1


In [15]:
training.columns

Index(['POSITIVE', 'Unnamed: 1', 'Unnamed: 2', 'Target'], dtype='object')

In [16]:
training.rename(
    columns = {'POSITIVE':'ID',
               'Unnamed: 1':'seq',
               'Unnamed: 2': 'site',
               'Target': 'Label'},
    inplace = True
)
training.drop('site', axis=1, inplace=True)
training.head(5)

,ID,seq,Label
0,P14543,ATTNGVVPADVILGTEDGAEYDDEDEDYDLATTRLGLEDVG,1
1,P14543,PADVILGTEDGAEYDDEDEDYDLATTRLGLEDVGTTPFSYK,1
2,P10493,TAKGVVSADVNLDLDDDGADYEDEDYDLVTSHLGLEDVATP,1
3,P10493,VSADVNLDLDDDGADYEDEDYDLVTSHLGLEDVATPSPSHS,1
4,P13521,ENKPYALNSEKNFPMDMSDDYETQQWPERKLKHMQFPPMYE,1


In [17]:
training.to_csv('cleaned_training.csv')

In [12]:
def getMatrixLabel(positive_position_file_name,sites, window_size=51, empty_aa = '*'):
    # input format   label, proteinName, postion,shortsequence,
    prot = []  # list of protein name
    pos = []  # list of position with protein name
    rawseq = []
    all_label = []

    short_seqs = []
    half_len = (window_size - 1) / 2

    with open(positive_position_file_name, 'r') as rf:
        reader = csv.reader(rf)
        for row in reader:

            position = int(row[2])
            sseq = row[3]
            rawseq.append(row[3])
            center = sseq[position - 1]
            if center in sites:
                all_label.append(int(row[0]))
                prot.append(row[1])
                pos.append(row[2])

                #short seq
                if position - half_len > 0:
                    start = position - half_len
                    left_seq = sseq[start - 1:position - 1]
                else:
                    left_seq = sseq[0:position - 1]

                end = len(sseq)
                if position + half_len < end:
                    end = position + half_len
                right_seq = sseq[position:end]

                if len(left_seq) < half_len:
                    nb_lack = half_len - len(left_seq)
                    left_seq = ''.join([empty_aa for count in range(nb_lack)]) + left_seq

                if len(right_seq) < half_len:
                    nb_lack = half_len - len(right_seq)
                    right_seq = right_seq + ''.join([empty_aa for count in range(nb_lack)])
                shortseq = left_seq + center + right_seq
                short_seqs.append(shortseq)
                # coding = one_hot_concat(shortseq)
                # all_codings.append(coding)

        targetY = kutils.to_categorical(all_label)

        ONE_HOT_SIZE = 21
        # _aminos = 'ACDEFGHIKLMNPQRSTVWY*'
        letterDict = {}
        letterDict["A"] = 0
        letterDict["C"] = 1
        letterDict["D"] = 2
        letterDict["E"] = 3
        letterDict["F"] = 4
        letterDict["G"] = 5
        letterDict["H"] = 6
        letterDict["I"] = 7
        letterDict["K"] = 8
        letterDict["L"] = 9
        letterDict["M"] = 10
        letterDict["N"] = 11
        letterDict["P"] = 12
        letterDict["Q"] = 13
        letterDict["R"] = 14
        letterDict["S"] = 15
        letterDict["T"] = 16
        letterDict["V"] = 17
        letterDict["W"] = 18
        letterDict["Y"] = 19
        letterDict["*"] = 20

        Matr = np.zeros((len(short_seqs), window_size, ONE_HOT_SIZE))
        samplenumber = 0
        for seq in short_seqs:
            AANo = 0
            for AA in seq:
                index = letterDict[AA]
                Matr[samplenumber][AANo][index] = 1
                AANo = AANo+1
            samplenumber = samplenumber + 1

    return Matr, targetY
